<a href="https://colab.research.google.com/github/arian-gif/Weather/blob/main/Weather_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weather RNN


## Importing Libraries

In [26]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

## Data Preprocessing

In [27]:
dataset = pd.read_csv('weather_data.csv')
y = dataset.iloc[:, 2].values

### Encoding the Date

In [28]:
dataset = dataset.drop(dataset.columns[2], axis=1)
dataset['Date_Time'] = pd.to_datetime(dataset['Date_Time'])

def encode_cyclical(data, max_val):
    sin = np.sin(2 * np.pi * data / max_val)
    cos = np.cos(2 * np.pi * data / max_val)
    return sin, cos

day_sin, day_cos = encode_cyclical(dataset['Date_Time'].dt.dayofyear, 366)
hour_sin, hour_cos = encode_cyclical(dataset['Date_Time'].dt.hour, 24)

dataset['day_of_year_sin'] = day_sin
dataset['day_of_year_cos'] = day_cos
dataset['hour_sin'] = hour_sin
dataset['hour_cos'] = hour_cos

dataset = dataset.drop(columns=['Date_Time'])
X = dataset.values

### Encoding categorical data


### One Hot Encoding the States


In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X= np.array(ct.fit_transform(X))

### Training and Test Set Split

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scale

In [31]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
y_train = sc_y.fit_transform(y_train.reshape(-1, 1))
y_test = sc_y.transform(y_test.reshape(-1, 1))

### Create Tensors

In [32]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)

y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_train_tensor = y_train_tensor.view(-1, 1).float()
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
y_test_tensor = y_train_tensor.view(-1, 1).float()


## Building the RNN

### Create the Model


In [33]:
class WeatherRNN(nn.Module):
  def __init__(self, input_size, rnn_hidden, hidden1, hidden2, output_size):
    super(WeatherRNN, self).__init__()
    self.rnn = nn.RNN(input_size, rnn_hidden, num_layers=2, batch_first=True)
    self.fc1 = nn.Linear(rnn_hidden, hidden1)
    self.tanh = nn.Tanh()
    self.fc2 = nn.Linear(hidden1, hidden2)
    self.fc_out = nn.Linear(hidden2, output_size)

  def forward(self, x):
    rnn_out, _ = self.rnn(x)
    last_step = rnn_out[:, -1, :]
    x = self.tanh(self.fc1(last_step))
    x = self.tanh(self.fc2(x))
    output = self.fc_out(x)
    return output


### Initalize the Model

In [34]:
num_classes = int(torch.max(y_train_tensor).item() + 1)
model = WeatherRNN(input_size=17, rnn_hidden=32, hidden1=64, hidden2=32, output_size=num_classes)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### Train the model

In [35]:
for epoch in range(32):
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/{32}], Loss: {loss.item():.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([800000, 1])) that is different to the input size (torch.Size([800000, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/32], Loss: 0.4350
Epoch [2/32], Loss: 0.4467
Epoch [3/32], Loss: 0.4279
Epoch [4/32], Loss: 0.4267
Epoch [5/32], Loss: 0.4274
Epoch [6/32], Loss: 0.4259
Epoch [7/32], Loss: 0.4242
Epoch [8/32], Loss: 0.4247
Epoch [9/32], Loss: 0.4256
Epoch [10/32], Loss: 0.4248
Epoch [11/32], Loss: 0.4240
Epoch [12/32], Loss: 0.4240
Epoch [13/32], Loss: 0.4242
Epoch [14/32], Loss: 0.4243
Epoch [15/32], Loss: 0.4243
Epoch [16/32], Loss: 0.4240
Epoch [17/32], Loss: 0.4238
Epoch [18/32], Loss: 0.4238
Epoch [19/32], Loss: 0.4237
Epoch [20/32], Loss: 0.4236
Epoch [21/32], Loss: 0.4236
Epoch [22/32], Loss: 0.4236
Epoch [23/32], Loss: 0.4235
Epoch [24/32], Loss: 0.4235
Epoch [25/32], Loss: 0.4235
Epoch [26/32], Loss: 0.4234
Epoch [27/32], Loss: 0.4233
Epoch [28/32], Loss: 0.4233
Epoch [29/32], Loss: 0.4232
Epoch [30/32], Loss: 0.4232
Epoch [31/32], Loss: 0.4232
Epoch [32/32], Loss: 0.4231


## Predictions

In [36]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)

preds_np = predictions.numpy()
y_pred_original =  sc_y.inverse_transform(preds_np)
y_test_original = sc_y.inverse_transform(y_test_tensor.numpy())

